# EXERCICE : ANONYMISATION DES DONNEES
# =================================================================

## 1. Redaction : Utilisons soit la généralisation ou la perturbation pour la rédaction des champs START, STOP, BIRTHDATE et DEATHDATE :

## 2. Agrégation : Résumons les données dans un nouveau tableau avec les exigences suivantes :

### 2.1. Age : agrégeons par chacun des déciles

In [1]:
# Importons les librairies nécessaires : 
import pandas as pd
import geopandas as gd
import matplotlib.pyplot as plt
import numpy as np

# Chargeons le fichier de données puis affichons les 5 premières lignes :
df = pd.read_csv("C:/Users/Kparatiogo/Desktop/GAVIN_BOYERA/patient-data-anonymisation-exercise.csv")
df.head(5)


Bad key "text.kerning_factor" on line 4 in
C:\Users\Kparatiogo\Anaconda3\envs\GEOPANDAS\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


,PATIENT_ID,START,STOP,ENCOUNTERCLASS,DESCRIPTION,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONDESCRIPTION,BIRTHDATE,DEATHDATE,...,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,034e9e3b-2def-4559-bb2a-7850888ae060,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
1,034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
2,034e9e3b-2def-4559-bb2a-7850888ae060,2015-01-26T17:45:28Z,2015-01-26T18:15:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
3,034e9e3b-2def-4559-bb2a-7850888ae060,2016-12-29T17:45:28Z,2016-12-29T18:00:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
4,034e9e3b-2def-4559-bb2a-7850888ae060,2017-01-09T17:45:28Z,2017-01-09T18:00:28Z,outpatient,Encounter for check up (procedure),129.16,54.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531


In [2]:
df = df.copy()

# Convertissons les champs de date en dates de pandas :

for c in ["START", "STOP", "BIRTHDATE", "DEATHDATE"]:
    df[c] = df[c].apply(lambda x: pd.to_datetime(x, errors="coerce", dayfirst=True).date())

In [3]:
# Importons la librairie datetime puis définissons une fontion année : 
import datetime

def annee(value):
    return value.year

df["BIRTHDATE"]=df["BIRTHDATE"].apply(annee)

In [4]:
# Afficher l'année de naissance uniquement
Date=df["BIRTHDATE"]
Date.head(10)

0    1983
1    1983
2    1983
3    1983
4    1983
5    1983
6    1989
7    1989
8    1989
9    1989
Name: BIRTHDATE, dtype: int64

### 2.2. Compte : pour chaque tranche d'âge, les raisons spécifiques de la visite à l'hôpital (de DESCRIPTION) et décider de la manière de présenter cela

In [5]:
grouped1 = df.groupby(["BIRTHDATE", 'DESCRIPTION'])
G1 = grouped1.aggregate(np.sum)
G1.head(30)

TOTAL_CLAIM_COST  \
BIRTHDATE DESCRIPTION                                                    
1909      Drug rehabilitation and detoxification                645.80   
          Emergency Room Admission                              774.96   
          Emergency room admission (procedure)                  129.16   
          Encounter Inpatient                                   232.47   
          Encounter for 'check-up'                              129.16   
          Encounter for check up (procedure)                    129.16   
          Encounter for problem (procedure)                    2066.56   
          Encounter for symptom                                 129.16   
          General examination of patient (procedure)           1679.08   
1911      Cardiac Arrest                                        129.16   
          Emergency Encounter                                  2324.88   
          Emergency Room Admission                              387.48   
          Emergency room admission (procedure)                 1679.08   
          Encounter for 'check-up'                              904.12   
          Encounter for Problem                                 387.48   
          Encounter for check up (procedure)                  22215.52   
          Encounter for problem                                7491.28   
          Encounter for symptom                                4520.60   
          Follow-up encounter                                  8653.72   
          General examination of patient (procedure)          27640.24   
          Inpatient stay (finding)                              258.32   
          Myocardial Infarction                                 129.16   
          Non-urgent orthopedic admission                       129.16   
          Office Visit                                          387.48   
          Stroke                                                258.32   
          Urgent care clinic (procedure)                      15628.36   
1913      Death Certification                                   129.16   
          Emergency Encounter                                 15628.36   
          Emergency Room Admission                              516.64   
          Emergency room admission (procedure)                  774.96   

                                                      PAYER_COVERAGE  \
BIRTHDATE DESCRIPTION                                                  
1909      Drug rehabilitation and detoxification              145.80   
          Emergency Room Admission                            174.96   
          Emergency room admission (procedure)                 29.16   
          Encounter Inpatient                                   0.00   
          Encounter for 'check-up'                             29.16   
          Encounter for check up (procedure)                    0.00   
          Encounter for problem (procedure)                   466.56   
          Encounter for symptom                                29.16   
          General examination of patient (procedure)         1459.08   
1911      Cardiac Arrest                                       69.16   
          Emergency Encounter                                 974.88   
          Emergency Room Admission                             54.16   
          Emergency room admission (procedure)                891.60   
          Encounter for 'check-up'                            534.96   
          Encounter for Problem                                89.16   
          Encounter for check up (procedure)                 8863.08   
          Encounter for problem                              3981.36   
          Encounter for symptom                              1819.00   
          Follow-up encounter                                5513.72   
          General examination of patient (procedure)        12841.44   
          Inpatient stay (finding)                              0.00   
          Myocardial Infarcti

In [6]:
G1.drop(G1.iloc[:,:],1,inplace=True)
G1.head(30)

Empty DataFrame
Columns: []
Index: [(1909, Drug rehabilitation and detoxification), (1909, Emergency Room Admission), (1909, Emergency room admission (procedure)), (1909, Encounter Inpatient), (1909, Encounter for 'check-up'), (1909, Encounter for check up (procedure)), (1909, Encounter for problem (procedure)), (1909, Encounter for symptom), (1909, General examination of patient (procedure)), (1911, Cardiac Arrest), (1911, Emergency Encounter), (1911, Emergency Room Admission), (1911, Emergency room admission (procedure)), (1911, Encounter for 'check-up'), (1911, Encounter for Problem), (1911, Encounter for check up (procedure)), (1911, Encounter for problem), (1911, Encounter for symptom), (1911, Follow-up encounter), (1911, General examination of patient (procedure)), (1911, Inpatient stay (finding)), (1911, Myocardial Infarction), (1911, Non-urgent orthopedic admission), (1911, Office Visit), (1911, Stroke), (1911, Urgent care clinic (procedure)), (1913, Death Certification), (1913, Emergency Encounter), (1913, Emergency Room Admission), (1913, Emergency room admission (procedure))]

### 2.3. Totals : pour chaque tranche d'âge, total de tous les coûts des sinistres

In [7]:
grouped2 = df.groupby(["BIRTHDATE", 'TOTAL_CLAIM_COST'])
G2 = grouped2.aggregate(np.sum)
G2.head(30)

PAYER_COVERAGE        ZIP            LAT  \
BIRTHDATE TOTAL_CLAIM_COST                                             
1909      77.49                       0.00     3603.0     127.379939   
          129.16                   2333.88    52844.0    1868.239105   
1911      129.16                  36221.64        0.0   29995.212899   
1913      129.16                 176996.64  4989126.0   99297.352856   
1914      77.49                      17.49        0.0      42.231339   
          129.16                  77188.32  1377033.0  114763.126513   
1915      129.16                  10160.00   275958.0    5374.659603   
1917      129.16                  69221.16  1941609.0   45676.034125   
1918      129.16                  18676.04   308535.0   10063.070525   
1920      129.16                  12806.48  1058936.0   21119.385592   
1922      129.16                  35477.12   913774.0   20686.050763   
1923      129.16                 150780.76   105633.0   83909.969148   
1924      129.16                   2101.52        0.0     927.460775   
1925      77.49                      17.49     2638.0      41.707110   
          129.16                  25597.64   511652.0   15478.614205   
1927      77.49                       0.00    10880.0     212.473548   
          129.16                  25565.56   686208.0   15746.793335   
1928      129.16                   2054.04    40147.0     803.557645   
1930      129.16                   3948.08   282984.0    4335.364044   
1933      77.49                       7.47        0.0     127.143947   
          129.16                  23776.48   171612.0   11921.997087   
1935      77.49                      17.49     6903.0     126.212743   
          129.16                  18342.60   815225.0   16592.506484   
1936      129.16                   1637.40        0.0     626.091499   
1937      77.49                      17.49     1475.0      42.717448   
          129.16                  62007.40  1364055.0   44867.871862   
1939      129.16                  64407.80   925847.0   41648.060929   
1940      77.49                      37.49        0.0      41.891337   
          129.16                  39283.40   460354.0   28864.708458   
1941      77.49                       7.49        0.0      42.512453   

                                      LON  
BIRTHDATE TOTAL_CLAIM_COST                 
1909      77.49               -219.929947  
          129.16             -3225.639222  
1911      129.16            -51026.597667  
1913      129.16           -166652.418268  
1914      77.49                -71.774596  
          129.16           -194596.274769  
1915      129.16             -9013.990992  
1917      129.16            -75701.465967  
1918      129.16            -17151.337929  
1920      129.16            -35497.290837  
1922      129.16            -34698.628900  
1923      129.16           -141695.410239  
1924      129.16             -1564.352135  
1925      77.49                -70.217039  
          129.16            -26047.016545  
1927      77.49               -355.276641  
          129.16            -26550.462295  
1928      129.16             -1349.920451  
1930      129.16             -7392.326311  
1933      77.49               -213.837995  
          129.16            -20041.903667  
1935      77.49               -213.033100  
          129.16            -27943.730455  
1936      129.16             -1067.506945  
1937      77.49                -72.053778  
          129.16            -75901.190699  
1939      129.16            -70372.380559  
1940      77.49                -70.854668  
          129.16            -48269.028078  
1941      77.49                -71.372751

In [8]:
G2.drop(G2.iloc[:,:],1,inplace=True)
G2.head(30)

Empty DataFrame
Columns: []
Index: [(1909, 77.49), (1909, 129.16), (1911, 129.16), (1913, 129.16), (1914, 77.49), (1914, 129.16), (1915, 129.16), (1917, 129.16), (1918, 129.16), (1920, 129.16), (1922, 129.16), (1923, 129.16), (1924, 129.16), (1925, 77.49), (1925, 129.16), (1927, 77.49), (1927, 129.16), (1928, 129.16), (1930, 129.16), (1933, 77.49), (1933, 129.16), (1935, 77.49), (1935, 129.16), (1936, 129.16), (1937, 77.49), (1937, 129.16), (1939, 129.16), (1940, 77.49), (1940, 129.16), (1941, 77.49)]

### 2.3. Médians : pour chaque tranche d'âge, la médiane de tous les coûts des sinistres

In [9]:
grouped3 = df.groupby(["BIRTHDATE", 'TOTAL_CLAIM_COST'])
G3 = grouped3.aggregate(np.median)
G3.head(30)

PAYER_COVERAGE     ZIP        LAT        LON
BIRTHDATE TOTAL_CLAIM_COST                                              
1909      77.49                       0.00  1201.0  42.459980 -73.309982
          129.16                     29.16  1201.0  42.459980 -73.309982
1911      129.16                     54.16     NaN  41.356635 -70.624884
1913      129.16                     89.16  2128.0  42.366926 -71.101551
1914      77.49                      17.49     NaN  42.231339 -71.774596
          129.16                      0.00  2062.0  42.249428 -71.733385
1915      129.16                     89.16  2170.0  42.322795 -70.976998
1917      129.16                     89.16  1950.0  42.791092 -70.848773
1918      129.16                     89.16  1535.0  42.269957 -72.083617
1920      129.16                     29.16  2126.0  42.319814 -71.166052
1922      129.16                     89.16  2140.0  42.413390 -71.127765
1923      129.16                     89.16  1089.0  42.390301 -71.172049
1924      129.16                    129.16     NaN  42.157308 -71.106915
1925      77.49                      17.49  2638.0  41.707110 -70.217039
          129.16                     69.16  2638.0  41.707110 -70.194525
1927      77.49                       0.00  2176.0  42.490773 -71.083913
          129.16                     89.16  2176.0  42.500614 -71.083913
1928      129.16                    129.16  2113.0  42.292508 -71.048445
1930      129.16                      0.00  2721.0  41.686193 -71.080061
1933      77.49                       2.49     NaN  42.381316 -71.279332
          129.16                     89.16  2043.0  42.210859 -70.928626
1935      77.49                       0.00  2301.0  42.070914 -71.011033
          129.16                     49.16  2301.0  42.070914 -71.011033
1936      129.16                    129.16     NaN  41.739433 -71.167130
1937      77.49                      17.49  1475.0  42.717448 -72.053778
          129.16                     69.16  2038.0  42.128572 -71.386084
1939      129.16                     69.16  2129.0  42.293097 -71.609261
1940      77.49                      37.49     NaN  41.891337 -70.854668
          129.16                     69.16  2458.0  42.345295 -70.854668
1941      77.49                       7.49     NaN  42.512453 -71.372751

In [10]:
G3.drop(G3.iloc[:,:],1,inplace=True)
G3.head(30)

Empty DataFrame
Columns: []
Index: [(1909, 77.49), (1909, 129.16), (1911, 129.16), (1913, 129.16), (1914, 77.49), (1914, 129.16), (1915, 129.16), (1917, 129.16), (1918, 129.16), (1920, 129.16), (1922, 129.16), (1923, 129.16), (1924, 129.16), (1925, 77.49), (1925, 129.16), (1927, 77.49), (1927, 129.16), (1928, 129.16), (1930, 129.16), (1933, 77.49), (1933, 129.16), (1935, 77.49), (1935, 129.16), (1936, 129.16), (1937, 77.49), (1937, 129.16), (1939, 129.16), (1940, 77.49), (1940, 129.16), (1941, 77.49)]